In [121]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from glob import glob
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import re

# スプレッドシート認証設定
SPREADSHEET_ID = "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, grape_calculator_myfive, assign_area):
    print(f'データ前処理を行います')
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre = df_pre[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calculator_myfive(df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    print(f'以下のモデルが含まれています')
    for i, model in enumerate(model_list):
        print(f'{i}: {model}', end=", ")
    return df_pre


def create_df_from_database(HALL_NAME, DB_PATH):
    # Table name 取得
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
            print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,     -- ホール名を追加
            m.name AS model_name     -- 機種名を追加
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
        JOIN models m ON r.model_id = m.model_id  -- 機種と結合
        WHERE h.name = ?  -- 指定ホールのみ
        AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
        ORDER BY r.date DESC, r.unit_no ASC;
        """

    df = pd.read_sql_query(query, conn, params=(hall_name,))
    conn.close()
    print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
    
    return df

In [122]:
HALL_NAME = "EXA FIRST"
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
df = create_df_from_database(HALL_NAME, DB_PATH)
df = df_preprocessing(df, grape_calculator_myfive, assign_area)
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 60648 x 11
データ前処理を行います
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-05-06,ゴーゴージャグラー3,1001,3781,14,13,-303,270.1,290.8,6.38,140.0,2025-05,6,1,1,a: 1001-
1,EXA FIRST,2025-05-06,ゴーゴージャグラー3,1002,6987,27,22,3,258.8,317.6,6.05,142.6,2025-05,6,1,2,a: 1001-
2,EXA FIRST,2025-05-06,ゴーゴージャグラー3,1003,1870,4,7,-726,467.5,267.1,6.20,170.0,2025-05,6,1,3,a: 1001-
3,EXA FIRST,2025-05-06,ゴーゴージャグラー3,1004,7252,29,35,735,250.1,207.2,6.50,113.3,2025-05,6,1,4,a: 1001-
4,EXA FIRST,2025-05-06,ゴーゴージャグラー3,1005,1236,5,1,-409,247.2,1236.0,6.83,206.0,2025-05,6,1,5,a: 1001-


## 出力データ
- 期間指定
    - 機種別の分析
    - 島別の分析
    - 台番号分析
    - 日付分析
    - 月分析
    - 過去n日の差枚比較
    - 全台系分析

## ピボットテーブル作成

In [123]:
def create_pivot_table(df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path, hall_name=HALL_NAME):
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=colmns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="total",
        )
        pivot_results[col] = table
    # pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals+game*3) / (game*3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([
        game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[interleaved_cols]
    merged.to_csv(csv_path)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate

## MEDAL_RATE

In [129]:
start_date = datetime.date.today()
# start_date = datetime.date(2024, 5, 6)
end_date = start_date - relativedelta(months=20, days=start_date.day - 1)
print(f"{start_date} to {end_date}")

csv_path = f"{HALL_NAME}_medal_rate.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
medal_rate.to_csv(csv_path)

rows, cols = medal_rate.shape
sheet_name = f"MEDAL_RATE"
try:
    worksheet = spreadsheet.worksheet(sheet_name)
    print(f"✅ シート「{sheet_name}」が既に存在します。")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
    print(f"🆕 シート「{sheet_name}」を新規作成しました。")
sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medal_rate, include_index=True)
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")
medal_rate.head(3)


2025-05-07 to 2023-09-01
✅ シート「MEDAL_RATE」が既に存在します。
✅ シート 'MEDAL_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
area     unit_no                                                               
a: 1001- 1001         1.037  0.993  1.013  1.000  0.993  0.968  1.000  0.958   
         1002         1.036  1.043  1.017  1.015  1.007  1.013  1.002  1.027   
         1003         1.047  1.025  1.035  0.990  1.050  0.957  1.004  1.013   

                                ...                                            \
day                   9     10  ...     23     24     25     26     27     28   
area     unit_no                ...                                             
a: 1001- 1001     1.005  0.951  ...  1.001  0.969  0.993  0.995  1.000  0.987   
         1002     0.990  1.035  ...  0.959  1.028  1.027  1.016  1.000  1.023   
         1003     0.979  0.978  ...  1.015  1.027  0.986  1.044  1.007  0.979   

                                              
day                  29     30     31  total  
area     unit_no                              
a: 1001- 1001     1.001  1.021  0.997  0.994  
         1002     0.998  1.012  1.011  1.012  
         1003     0.986  0.992  0.959  1.014  

[3 rows x 32 columns]

## 島分析

In [81]:
csv_path = f"{HALL_NAME}_area.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
merged.head(3)


,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,...,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE
day,1,1,1,1,1,1,1,2,2,2,...,31,31,31,total,total,total,total,total,total,total
area,,,,,,,,,,,,,,,,,,,,,
a: 1001-,847467,74604,3386,2619,1.029,323.6,141.1,604290,20762,2377,...,1.011,335.5,147.2,18504477,492827,70700,54453,1.009,339.8,147.9
b: 1011-,836350,38935,3224,2598,1.016,321.9,143.7,629433,10367,2370,...,1.007,323.6,146.4,18756774,493816,71503,55715,1.009,336.7,147.4
c: 1021-,524884,54843,2089,1676,1.035,313.2,139.4,389611,-3323,1451,...,0.995,372.0,154.9,11461737,252159,43558,33468,1.007,342.5,148.8


## 台分析

In [82]:
csv_path = f"{HALL_NAME}_unit.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
merged.head(3)



GAME MEDALS   BB   RB MEDAL_RATE RB_RATE TOTAL_RATE   GAME  \
day                   1      1    1    1          1       1          1      2   
area     unit_no                                                                
a: 1001- 1001     93043  10197  386  270      1.037   344.6      141.8  66525   
         1002     84120   9100  328  298      1.036   282.3      134.4  57413   
         1003     89103  12669  369  289      1.047   308.3      135.4  72837   

                 MEDALS   BB  ... MEDAL_RATE RB_RATE TOTAL_RATE     GAME  \
day                   2    2  ...         31      31         31    total   
area     unit_no              ...                                          
a: 1001- 1001     -1300  247  ...      0.997   313.4      146.7  2062093   
         1002      7361  247  ...      1.011   389.5      150.7  1784285   
         1003      5408  291  ...      0.959   344.7      163.0  1767779   

                 MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  
day               total total total      total   total      total  
area     unit_no                                                   
a: 1001- 1001    -35093  7642  5856      0.994   352.1      152.8  
         1002     63798  6822  5301      1.012   336.6      147.2  
         1003     74583  6812  5361      1.014   329.7      145.2  

[3 rows x 224 columns]

# 日付、台番号指定

In [ ]:
def create_pivot_table_date(
    start_date, end_date, day_targets, pivot_targets, index_targets, colmns_targets
):
    """日付ごとにシートを作成して台ごとの出玉率一覧を作成"""
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    df_filtered = df_filtered[df_filtered["day"] == day_targets]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=colmns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="Total",
        )
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate



start_date = datetime.date.today()
end_date = start_date - relativedelta(months=20, days=start_date.day - 1)
print(f"{start_date} to {end_date}")

pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["date"]

day_target = 12
for day_target in range(20, 32):
    merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table_date(
        start_date, end_date, day_target, pivot_targets, index_targets, colmns_targets
    )

    rows, cols = medal_rate.shape
    sheet_name = f"DAY{day_target}"
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    set_with_dataframe(sheet, medal_rate, include_index=True)
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

medal_rate.head()

2025-05-07 to 2023-09-01
✅ シート「DAY1」が既に存在します。
✅ シート 'DAY1' に DataFrame を書き込みました！
✅ シート「DAY2」が既に存在します。
✅ シート 'DAY2' に DataFrame を書き込みました！
✅ シート「DAY3」が既に存在します。
✅ シート 'DAY3' に DataFrame を書き込みました！
✅ シート「DAY4」が既に存在します。
✅ シート 'DAY4' に DataFrame を書き込みました！
✅ シート「DAY5」が既に存在します。
✅ シート 'DAY5' に DataFrame を書き込みました！
✅ シート「DAY6」が既に存在します。
✅ シート 'DAY6' に DataFrame を書き込みました！
✅ シート「DAY7」が既に存在します。
✅ シート 'DAY7' に DataFrame を書き込みました！
✅ シート「DAY8」が既に存在します。
✅ シート 'DAY8' に DataFrame を書き込みました！
✅ シート「DAY9」が既に存在します。
✅ シート 'DAY9' に DataFrame を書き込みました！
✅ シート「DAY10」が既に存在します。
✅ シート 'DAY10' に DataFrame を書き込みました！
✅ シート「DAY11」が既に存在します。
✅ シート 'DAY11' に DataFrame を書き込みました！
✅ シート「DAY12」が既に存在します。
✅ シート 'DAY12' に DataFrame を書き込みました！
✅ シート「DAY13」が既に存在します。
✅ シート 'DAY13' に DataFrame を書き込みました！
✅ シート「DAY14」が既に存在します。
✅ シート 'DAY14' に DataFrame を書き込みました！
✅ シート「DAY15」が既に存在します。
✅ シート 'DAY15' に DataFrame を書き込みました！
✅ シート「DAY16」が既に存在します。
✅ シート 'DAY16' に DataFrame を書き込みました！
✅ シート「DAY17」が既に存在します。
✅ シート 'DAY17' に DataFrame を書き込みました！
✅ シート「D

APIError: APIError: [429]: Quota exceeded for quota metric 'Write requests' and limit 'Write requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:660712279213'.

In [ ]:
# 各種ピボットテーブル
def create_pivot_tabel(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    pivot_targets = ["game", "medals", "RB_rate", "Total_rate", "BB", "RB"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=["model_name", "area", "unit_no"],
            columns="day",
            values=col,
            aggfunc="sum",
        )
        # 日付列を反転・スライス
        # pivot_results[col] = table
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]

    return merged, game, medals, bb, rb, rb_rate, total_rate


start_date = datetime.date.today()
# start_date = datetime.date(2024, 5, 6)
end_date = start_date - relativedelta(days=30)
print(f"{start_date} to {end_date}")

merged, game, medals, bb, rb, rb_rate, total_rate = create_pivot_tabel(
    df, "マイジャグラーV", start_date, end_date
)
merged = merged[~(merged.iloc[:, 0].isna() | (merged.iloc[:, 0] == 0))]
merged.head()

2025-05-05 to 2025-04-05


GAME  MEDALS    BB    RB RB_RATE TOTAL_RATE  \
day                          1       1     1     1       1          1    
model_name area unit_no                                                  
マイジャグラーV   c    1021     6803.0 -1229.0  20.0  22.0   309.2      162.0   
                1022     8704.0  5206.0  49.0  36.0   241.8      102.4   
                1023     7751.0  1488.0  33.0  24.0   323.0      136.0   
                1024     6661.0  1544.0  27.0  24.0   277.5      130.6   
                1025     7689.0  2268.0  35.0  24.0   320.4      130.3   

                           GAME  MEDALS    BB    RB  ...    BB    RB RB_RATE  \
day                          2       2     2     2   ...    29    29      29   
model_name area unit_no                              ...                       
マイジャグラーV   c    1021     6785.0 -1453.0  18.0  21.0  ...  24.0  20.0   338.5   
                1022     2233.0 -1076.0   4.0   6.0  ...  23.0   9.0   717.1   
                1023     3522.0  -756.0  11.0   8.0  ...  24.0  19.0   307.5   
                1024     2461.0  -759.0   7.0   6.0  ...   3.0   1.0  1303.0   
                1025     1178.0 -1088.0   2.0   0.0  ...  29.0  19.0   367.6   

                        TOTAL_RATE    GAME  MEDALS    BB    RB RB_RATE  \
day                             29      30      30    30    30      30   
model_name area unit_no                                                  
マイジャグラーV   c    1021         153.9  6977.0 -2603.0  14.0  23.0   303.3   
                1022         201.7  5396.0  -903.0  19.0  12.0   449.7   
                1023         135.9   927.0  -456.0   2.0   2.0   463.5   
                1024         325.8  7662.0  1738.0  31.0  31.0   247.2   
                1025         145.5  3007.0  -468.0  10.0   9.0   334.1   

                        TOTAL_RATE  
day                             30  
model_name area unit_no             
マイジャグラーV   c    1021         188.6  
                1022         174.1  
                1023         231.8  
                1024         123.6  
                1025         158.3  

[5 rows x 180 columns]

In [90]:
model_list = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
    # "ミスタージャグラー",
    # "ウルトラミラクルジャグラー",
    # "ハッピージャグラーVIII",
    # "ジャグラーガールズ",
]

start_date = datetime.date.today()
end_date = start_date - relativedelta(days=60)
print(f"{start_date} to {end_date}")

merged_models = pd.DataFrame()
for model in list(model_list):
    print(model)
    merged, game, medals, bb, rb, rb_rate, total_rate = create_pivot_tabel(df, model, start_date, end_date)
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        area_merged = area_merged[~area_merged.iloc[:, 0].isna()]
        empty_index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
        empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
        
        # if area_merged.shape[0]:
        #     summary_row = []
        #     for c in area_merged.columns:
        #         if c[0] in ["GAME", "MEDALS", "RB", "BB"]:
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             summary_row.append(area_merged[c].sum())
        #         elif c[0] == "RB_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             rb_rate = game_sum / rb_sum if rb_sum != 0 else None
        #             summary_row.append(round(rb_rate, 1) if rb_rate else None)
        #         elif c[0] == "TOTAL_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             bb_sum = area_merged.get(("BB", c[1]), pd.Series()).sum()
        #             total_rate = game_sum / (rb_sum + bb_sum) if (rb_sum + bb_sum) != 0 else None
        #             summary_row.append(round(total_rate, 1) if total_rate else None)
            

            # summary_df = pd.DataFrame(summary_row).T
            # summary_df.index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
            # summary_df.columns = area_merged.columns
        merged_models = pd.concat([merged_models, area_merged, empty_row])

merged_models = merged_models[~(merged_models.iloc[:, 0].isna() | (merged_models.iloc[:, 0] == 0))]
merged_models.to_csv(f"{HALL_NAME}_unit.csv")
    
merged_models.head(10)

2025-05-05 to 2025-03-06
マイジャグラーV


GAME MEDALS  BB  RB RB_RATE TOTAL_RATE  GAME MEDALS  \
day                        1      1   1   1       1          1     2      2    
model_name area unit_no                                                        
マイジャグラーV   a    1001     6879    344  24  24   286.6      143.3  5376    838   
                1002     1441   -450   5   3   480.3      180.1  3486    221   
                1003     7691   3306  42  18   427.3      128.2  1224   -326   
                1004     4512    226  19  11   410.2      150.4   862   -426   
                1005     1852   -162   9   1  1852.0      185.2  6749   2621   
                1006     3912   -662  13  10   391.2      170.1  1346    150   
                1007     3854    -15  15   7   550.6      175.2  1940    268   
                1008     4801   -874  16   8   600.1      200.0  1897   -650   
                1009     6071    276  23  16   379.4      155.7  1224   -709   
                1010     5913   1100  26  18   328.5      134.4  3672   -544   

                         BB  RB  ...  BB  RB RB_RATE TOTAL_RATE  GAME MEDALS  \
day                      2   2   ...  30  30      30         30    31     31   
model_name area unit_no          ...                                           
マイジャグラーV   a    1001     26  13  ...  12   6   486.7      162.2  7407    865   
                1002     12  18  ...  29  32   262.2      137.5  6947   2003   
                1003      4   2  ...   6   4   388.2      155.3  3610   -503   
                1004      2   2  ...   8   9   350.6      185.6  1052   -438   
                1005     34  25  ...  24  15   413.5      159.1  9461   3612   
                1006      5   5  ...  24  21   348.4      162.6  1037   -621   
                1007      9   4  ...  18   5   968.6      210.6  3913   -462   
                1008      6   2  ...   7   9   312.0      175.5  6506    124   
                1009      3   3  ...   7   4   485.2      176.5  6350   -253   
                1010     14   6  ...  17  17   323.2      161.6  2981  -1429   

                         BB  RB RB_RATE TOTAL_RATE  
day                      31  31      31         31  
model_name area unit_no                             
マイジャグラーV   a    1001     27  33   224.5      123.4  
                1002     32  26   267.2      119.8  
                1003     11  10   361.0      171.9  
                1004      3   1  1052.0      263.0  
                1005     44  40   236.5      112.6  
                1006      2   1  1037.0      345.7  
                1007     14   9   434.8      170.1  
                1008     24  21   309.8      144.6  
                1009     24  14   453.6      167.1  
                1010      5   9   331.2      212.9  

[10 rows x 186 columns]

In [6]:
len(area_merged.columns)

180